<h5> test of download_model_function </h5>


In [3]:
from work_with_model import download_and_save_model
model = "sentence-transformers/paraphrase-albert-small-v2"

model.replace("@","@")

'sentence-transformers/paraphrase-albert-small-v2'

In [2]:
from work_with_model import load_model_bin

model_name = "sentence-transformers/paraphrase-MiniLM-L3-v2"
# model_name=model.split("/")[-1]
print(model_name)
obj = load_model_bin(model_name)

sentence-transformers/paraphrase-MiniLM-L3-v2


In [3]:
obj[1]

BertTokenizerFast(name_or_path='models/sentence-transformers/paraphrase-MiniLM-L3-v2/tokenizer', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [2]:
from work_with_model import load_model_bin

load_model_bin()

False


(BertForSequenceClassification(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(30522, 384, padding_idx=0)
       (position_embeddings): Embedding(512, 384)
       (token_type_embeddings): Embedding(2, 384)
       (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0-2): 3 x BertLayer(
           (attention): BertAttention(
             (self): BertSelfAttention(
               (query): Linear(in_features=384, out_features=384, bias=True)
               (key): Linear(in_features=384, out_features=384, bias=True)
               (value): Linear(in_features=384, out_features=384, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): BertSelfOutput(
               (dense): Linear(in_features=384, out_features=384, bias=True)
               (LayerNorm): LayerNor

In [10]:
from work_with_model import encode_model_bin
import torch

def encode_text(text):
    model,tokenizer = load_model_bin()
    print(model is None)

    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        return_token_type_ids=False,
        return_attention_mask=False,
        return_tensors='pt'
    )
    with torch.no_grad():
        outputs = model(**encoding)
        last_hidden_state = outputs.hidden_states[-1]
        embeddings = last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()


In [12]:
encode_text("nlp book")

False


TypeError: 'NoneType' object is not subscriptable

In [6]:
from dataclasses import dataclass

@dataclass
class model:
    a = 1
    b = a+1

p = model()

In [8]:
p.b

2

## new test

In [2]:
from work_with_model import transformer_ops as tops
name = "GPT_125M"
wom = tops(name=name)
# name = "sentence-transformers@paraphrase-albert-small-v2"

In [3]:
wom.setter(name)

Directory already exists


In [4]:
wom.download_and_save_model_pickle(name)

GPT_125M already exists no need to download again ...


In [9]:
vec = wom.encode_single_doc("the cat is strong and has eaten rat")

In [10]:
vec

array([-0.08235384, -0.02042743,  0.06621283, -0.00539877,  0.04291169,
       -0.05068491, -0.03325076,  0.0922316 ,  0.00734016,  0.01798994,
       -0.0713305 ,  0.05039785, -0.01601499,  0.1057223 ,  0.02131992,
        0.10951817,  0.04903874, -0.02486438,  0.01305457,  0.11004685,
        0.03038722,  0.00421404,  0.03267882, -0.049477  ,  0.03422258,
        0.10292463,  0.01380672,  0.06906645,  0.08547013,  0.10240253,
        0.02345021,  0.06812023, -0.0618082 , -0.00498297, -0.05749562,
       -0.07924826, -0.06724481,  0.05553465, -0.01387   ,  0.03890328,
        0.03713493,  0.10886794, -0.07062145,  0.0373233 , -0.01620992,
       -0.08771899, -0.11835511, -0.08457901,  0.01721183, -0.0514229 ,
       -0.0351016 ,  0.03031924,  0.08902972, -0.0394065 , -0.1025086 ,
       -0.0105591 ,  0.00085191,  0.00470831, -0.00350298, -0.03778702,
       -0.01487388,  0.00477025,  0.02340586, -0.02283034,  0.01276157,
       -0.11446246, -0.08243497, -0.13390358,  0.00978376,  0.00

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-albert-small-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-albert-small-v2')


def encode_into_vector():
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    print("Sentence embeddings:")
    print(sentence_embeddings)
